In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files, drive
import pandas as pd
import numpy as np

# Mount Google Drive
drive.mount('/content/gdrive')

uploaded = files.upload()

# Use 'results.csv' instead of 'LSTMData.csv'
LSTMData = pd.read_csv('FixedTrainData.csv')

# These cleanup steps aren't required anymore based on the format we created:
# LSTMData = LSTMData[~LSTMData['prev_cluster_seq'].str.contains('NA')]
# LSTMData = LSTMData.dropna(subset=['next_cluster'])

# Update column names to match the dataset's structure
LSTMData['sequence'] = LSTMData['sequence'].apply(lambda x: [int(i) for i in x.strip("[]").split(",")])

def create_model():
    model = Sequential()
    model.add(LSTM(100, input_shape=(10, 1)))
    model.add(Dense(6, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Use 'speaker_id' instead of 'next_cluster_speaker_id'
speaker_ids = LSTMData['speaker_id'].unique()

results = []
counter = 1
for speaker_id in speaker_ids:
    train_data = LSTMData[LSTMData['speaker_id'] != speaker_id]
    test_data = LSTMData[LSTMData['speaker_id'] == speaker_id]

    X_train = np.array(train_data['sequence'].tolist()).reshape(-1, 10, 1)
    y_train = to_categorical(train_data['target'] - 1, num_classes=6)

    X_test = np.array(test_data['sequence'].tolist()).reshape(-1, 10, 1)
    y_test = to_categorical(test_data['target'] - 1, num_classes=6)

    model = create_model()

    print(f"Running Training Session {counter} for Speaker ID {speaker_id}")
    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    score = model.evaluate(X_test, y_test, verbose=1)
    print(f'Accuracy for speaker {speaker_id}: {score[1]*100}%')

    results.append({'speaker_id': speaker_id, 'score': score[1]})

    predictions = model.predict(X_test)
    pred_df = pd.DataFrame({
        'input_sequence': test_data['sequence'].tolist(),
        'predicted_cluster': np.argmax(predictions, axis=1) + 1,
        'actual_cluster': np.argmax(y_test, axis=1) + 1,
        'seg_id': test_data['seg_id'].tolist(),
        'speaker_id': speaker_id
    })

    pred_df.to_csv(f'/content/gdrive/My Drive/FixedLSTMPredictions{speaker_id}.csv', index=False)

    counter += 1


Mounted at /content/gdrive


Saving FixedTrainData.csv to FixedTrainData.csv
Running Training Session 1 for Speaker ID B-01
Epoch 1/10
790/790 [==============================] - 11s 11ms/step - loss: 0.7515 - accuracy: 0.7545
Epoch 2/10
790/790 [==============================] - 8s 10ms/step - loss: 0.7324 - accuracy: 0.7554
Epoch 3/10
790/790 [==============================] - 8s 11ms/step - loss: 0.7260 - accuracy: 0.7554
Epoch 4/10
790/790 [==============================] - 8s 10ms/step - loss: 0.7217 - accuracy: 0.7554
Epoch 5/10
790/790 [==============================] - 8s 11ms/step - loss: 0.7179 - accuracy: 0.7554
Epoch 6/10
790/790 [==============================] - 8s 10ms/step - loss: 0.7117 - accuracy: 0.7554
Epoch 7/10
790/790 [==============================] - 8s 11ms/step - loss: 0.6989 - accuracy: 0.7553
Epoch 8/10
790/790 [==============================] - 8s 10ms/step - loss: 0.6867 - accuracy: 0.7555
Epoch 9/10
790/790 [==============================] - 8s 10ms/step - loss: 0.6785 - accuracy: 0.